In [1]:
import pandas as pd
from datasets import Dataset
import numpy as np
import torch
from transformers import AutoTokenizer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
from scipy.spatial.distance import cosine
from sentence_transformers import SentenceTransformer
import evaluate

c:\Anaconda\envs\Cuda\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Anaconda\envs\Cuda\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\Anaconda\envs\Cuda\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
c:\Anaconda\envs\Cuda\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
Train_Data_Frame = pd.read_json('../Datasets/AIC Val/ds.jsonl', lines=True)
Train_Data_Frame

,example_id,paragraph,summary
0,0,وتحت عنوان من الكارثة إلى التحدى يبدأ الكاتب ع...,يبدأ الكاتب عرض الكتاب الرابع تحت عنوان من الك...
1,1,ولم يعترف دبلوماسيو هاتين الدولتين بالعريضة ال...,دبلوماسيو الدولتين لم يعترفوا بالعريضة التي قا...
2,2,قامت ولاية حلب بعد اعلان الجنرال الفرنسي هنري ...,أعلن غورو الانتداب الفرنسي على سوريا لكي يعاقب...
3,3,دولة مصر العربيه هي ليست اي دوله وليست اي شعب ...,مصر هي أم البلاد، وقائدة العرب؛ فهي أرض بلاد ا...
4,4,السوريون يصرون على استقلال بلادهم : و مثلما رف...,الشعب السوري يصر على استقلال بلدهم من السيطرة ...
...,...,...,...
149,149,حزب الوفد سيحتفل بمئوية ثورة 1919 يوم 9 مارس ا...,احتفال مئوية ثورة 1919 كان من منطلق وطني ليس ح...
150,150,حيث أعلن مجلس قيادة الثورة في 18 يونيه 1953 قي...,مجلس قيادة الثورة أعلن عن قيام الجمهورية المصر...
151,151,وبرغم أن عبد الرحمن فهمي كان يضم في ذلك الجهاز...,ضم عبد الرحمن فهمي في الجهاز السري عدد كبير من...
152,152,ولم تقتصر مقومات بورسعيد كمدينة عالمية منذ نشأ...,امتدت بورسعيد لكي تشمل الطابع الثقافي للمدينة،...


In [3]:
Test_Data_Frame = pd.read_csv('../Datasets/Combined Dataset/ModCombined.csv')
Test_Data_Frame

,text,summary
0,داعش قالت إنها عازمة على الزحف نحو العاصمة الع...,أفادت الأنباء الواردة من العراق بحدوث تقدم قام...
1,#ماذا_تعرف_عن_الاقصى لا تزال أصداء ما جري في م...,استحوذت علامات التجزئة العديدة على اهتمام المغ...
2,وخضع دي ماريا (27 عاما) للكشف الطبي في قطر يوم...,أعرب الأرجنتيني أنخيل دي ماريا عن ارتياحه الكب...
3,وأوضح سكان محليون أن ميليشيات مرتبطة بجماعة أن...,لسوء الحظ ، وقع حادث مأساوي في مدينة بنغازي ، ...
4,اشتباكات في حلب السورية وذكرت الهيئة العامة لل...,بحسب ناشطين ، أفادت الأنباء أن خمسة أفراد فقدو...
...,...,...
701,ترامب أثناء تأديته قسم تولي رئاسة الولايات الم...,أعرب الرئيس الأمريكي الجديد دونالد ترامب ، خلا...
702,"هوية المصور العسكري ""قيصر"" المنشق عن النظام ال...","وافق مجلس النواب الأمريكي بالإجماع على ""قانون ..."
703,دأب سكولز على انتقاد أداء يونايتيد في المواسم ...,وفقًا لبول سكولز ، نجم مانشستر يونايتد السابق ...
704,كسوف كلي للشمس في أستراليا تجمع عشرات الألاف م...,اجتمع عدد كبير من السياح وعلماء الفلك في شمال ...


In [4]:
Train_Data_Frame = Dataset.from_pandas(Train_Data_Frame)
Test_Data_Frame = Dataset.from_pandas(Test_Data_Frame)

In [5]:
Train_Dataset_Paragraphs_Column_Name = 'paragraph'
Train_Dataset_Summaries_Column_Name = 'summary'

Test_Dataset_Paragraphs_Column_Name = 'text'
Test_Dataset_Summaries_Column_Name = 'summary'

In [6]:
Train_Data_Frame = Train_Data_Frame.train_test_split(test_size=0.2)

In [7]:
Train_Data_Frame["train"][0]

{'example_id': 121,
 'paragraph': 'مهما يكن فإن المدينة تاريخيا ، واحدة من أقدم المواقع المسكونة في منطقة الشرق الأوسط ، وربما يعود تاريخ استيطانها البشري إلى أكثر من 9500 سنة ، أي إلى العصر الحجري الحديث الذي شهد الثورة الزراعية في غرب آسيا ومنطقة الهلال الخصيب أو المشرق العربي وقبرص . وقد تم العثور على آثار وتحف في المقابر يعود تاريخها إلى ثلاثة آلاف سنة . وعام 1500 قبل الميلاد وقعت المدينة تحت سيطرة الحثيين . ويبدو أن المدينة في القرن الثالث قبل الميلاد كانت من أهم المحطات الثقافية والتجارية المتقدمة في منطقة الأناضول . وبعد غزو أهل البحر الفينيقيين من فلسطين ولبنان وأهالي دول البلقان سيطر الإغريق على المدينة . ويقال أيضا إن هومر الشاعر اليوناني المعروف قديما وصاحب الأوديسة والإلياذة من أبناء المدينة . ويقول هيرودوتس إن المدينة أسست من قبل الألونيين قبل أن يسيطر عليها الأيونيون الإغريق في القرن السابع قبل الميلاد . ويقال أيضا إن بعد تلك الفترة بدأت المدينة تتمتع بصفتها كمدينة ، على الرغم من أن عدد سكانها لم يتعد الألف نسمة آنذاك . وكانت محاطة ببساتين الزيتون والعنب ومصانع الأدوات ال

In [8]:
Model_Name = "facebook/mbart-large-50"
Tokenizer = AutoTokenizer.from_pretrained(Model_Name)
Model = AutoModelForSeq2SeqLM.from_pretrained(Model_Name)
Semantic_Model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

In [9]:
Prefix = "summarize: "
Paragraph_Max_Length = 1000
Summary_Max_Length = 300

In [10]:
def Tokenization(Records):
    Paragraphs = [Prefix + Paragraph for Paragraph in Records[Train_Dataset_Paragraphs_Column_Name]]
    Final_Tokenized_Inputs = Tokenizer(Paragraphs, max_length=1000, truncation=True, padding="longest", return_tensors="pt")

    Labels = Tokenizer(Records[Train_Dataset_Summaries_Column_Name], max_length=300, truncation=True, padding="longest", return_tensors="pt")

    Final_Tokenized_Inputs["labels"] = Labels["input_ids"]
    return Final_Tokenized_Inputs

In [11]:
Tokenized_Dataset = Train_Data_Frame.map(Tokenization, batched=True)

Map:   0%|          | 0/123 [00:00<?, ? examples/s]

In [12]:
Data_Collator = DataCollatorForSeq2Seq(tokenizer=Tokenizer, model=Model_Name)

In [13]:
Rouge = evaluate.load("rouge")

In [14]:
def Compute_Metrics(Eval_Pred):
    Predictions, Labels = Eval_Pred
    Decoded_Preds = Tokenizer.batch_decode(Predictions, skip_special_tokens=True)
    Labels = np.where(Labels != -100, Labels, Tokenizer.pad_token_id)
    Decoded_Labels = Tokenizer.batch_decode(Labels, skip_special_tokens=True)

    similarity_scores = []
    for pred, label in zip(Decoded_Preds, Decoded_Labels):
        pred = pred[len(Prefix):]
        embeddings = Semantic_Model.encode([pred, label], convert_to_tensor=True)
        similarity_score = 1 - cosine(embeddings[0].cpu(), embeddings[1].cpu())
        similarity_scores.append(similarity_score)

    Result = Rouge.compute(predictions=Decoded_Preds, references=Decoded_Labels, use_stemmer=True)

    Prediction_Lens = [np.count_nonzero(Pred != Tokenizer.pad_token_id) for Pred in Predictions]
    Result["gen_len"] = np.mean(Prediction_Lens)
    Result["semantic_similarity"] = np.mean(similarity_scores)

    print("Decoded Predictions:", Decoded_Preds)
    print("Decoded Labels:", Decoded_Labels)

    return {k: round(v, 4) for k, v in Result.items()}

In [15]:
training_args = Seq2SeqTrainingArguments(
    output_dir="Check_Points_Output",
    evaluation_strategy="epoch",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=4,
    predict_with_generate=True,
)

trainer = Seq2SeqTrainer(
    model=Model,
    args=training_args,
    train_dataset=Tokenized_Dataset["train"],
    eval_dataset=Tokenized_Dataset["test"],
    tokenizer=Tokenizer,
    data_collator=Data_Collator,
    compute_metrics=Compute_Metrics,
)

In [18]:
trainer.train()

c:\Anaconda\envs\Cuda\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
 22%|██▏       | 500/2256 [46:06<2:58:13,  6.09s/it]

{'loss': 1.9006, 'learning_rate': 3.905141843971632e-05, 'epoch': 0.89}


                                                    
 25%|██▌       | 564/2256 [1:19:36<2:51:50,  6.09s/it]

Decoded Predictions: ['أعربت هيئة الأرصاد الجوية المصرية عن مخاوفها من أن "الإعصار المزيّف" المزعوم قد يضرب مصر والأراضي الفلسطينية. بالإضافة إلى ذلك ، أعربت هيئة الأرصاد الجوية المصرية عن مخاوفها من أن "الإعصار المزيّف" قد يشكل خطرًا كبيرًا على البنية التحتية المصرية.', 'أعرب رئيس الوزراء التركي ، أحمد داود أوغلو ، عن مخاوفه من أن العمليات العسكرية التركية ضد حزب العمال الكردستاني ، التي يقودها حزب العدالة والتنمية ، لن تكون محدودة. ومع ذلك ، أشار أوغلو إلى أن تركيا ، التي تتعامل حاليًا مع هجمات تنظيم الدولة الإسلامية ، قد لا تمتلك القدرة الكاملة على السيطرة الكاملة على هجمات التنظيم.', 'أصدرت محكمة جنح مدينة نصر ، بتهمة ازدراء الأديان مع الشغل ، حكمًا نهائيًا بعقوبة الحبس لمدة 10 سنوات. بالإضافة إلى ذلك ، أصدرت محكمة جنح مدينة نصر قرارًا بوقف تنفيذ الحكم المؤقت الصادر عن محكمة جنح مدينة نصر بتهمة ازدراء الأديان.', 'بحسب وكالة المخابرات المركزية (بي بي سي) ، فقد أفادت وكالة المخابرات المركزية (بي بي سي) أن أكثر من 40 بالمئة من مخزون سوريا من الأسلحة الكيمياوية المكتشفة خارج البلاد قد 

 44%|████▍     | 1000/2256 [1:57:55<1:51:31,  5.33s/it]  

...

 89%|████████▊ | 2000/2256 [4:15:40<23:51,  5.59s/it]    

{'loss': 0.3287, 'learning_rate': 5.806737588652482e-06, 'epoch': 3.55}


                                                     
100%|██████████| 2256/2256 [5:10:16<00:00,  8.25s/it]

Decoded Predictions: ['استحوذت عدة مخاوف على اهتمام مستخدمي وسائل التواصل الاجتماعي في العالم العربي ، بما في ذلك مخاوف "الإعصار المداري" المزعوم الذي يضرب مصر والأراضي الفلسطينية. هيئة الأرصاد الجوية المصرية توقعت حدوث إعصار في مصر يوم السبت. هيئة الأرصاد الجوية المصرية تراقب الوضع بدقة وتشمل الأمطار الغزيرة والرعدية.', 'فيما يتعلق بالعمليات العسكرية التركية ضد حزب العمال الكردستاني ، يُنظر إليها عمومًا على أنها تهدف إلى تخفيف المخاوف بين الأكراد الذين يحتجون على وجود تنظيم الدولة الإسلامية ، فضلاً عن استهداف مواقع لأفراد يشتبه في انتمائهم لتنظيم الدولة الإسلامية.', 'أصدرت محكمة جنح مدينة نصر مصرية حكما بالسجن 15 عامًا بتهمة ازدراء الأديان وتكدير الأمن العام. بالإضافة إلى ذلك ، قضت المحكمة بغرامة تصل إلى خمسمائة جنيه لكل من أبو إسلام ، وهاني محمد ياسين ، صحفي قناة الأمة الفضائية والمدون السابق بكلية الشريعة والقانون.', 'بحسب الأمم المتحدة ، فإن الحكومة السورية ، التي تتعامل حاليًا مع وضع أمني صارم ، قد تضطر إلى تأجيل النقل المقرر للمواد الكيمياوية داخل البلاد. يأتي هذا القرار نتيجة تد

TrainOutput(global_step=2256, training_loss=0.8349473036772815, metrics={'train_runtime': 18616.798, 'train_samples_per_second': 0.121, 'train_steps_per_second': 0.121, 'train_loss': 0.8349473036772815, 'epoch': 4.0})

In [133]:
def Calculate_Metrics(Predictions, Labels):
    Decoded_Preds = Tokenizer.batch_decode(Predictions, skip_special_tokens=True)
    Labels = np.where(Labels != -100, Labels, Tokenizer.pad_token_id)
    Decoded_Labels = Tokenizer.batch_decode(Labels, skip_special_tokens=True)

    similarity_scores = []
    for pred, label in zip(Decoded_Preds, Decoded_Labels):
        pred = pred[len(Prefix):]
        embeddings = Semantic_Model.encode([pred, label], convert_to_tensor=True)
        similarity_score = 1 - cosine(embeddings[0].cpu(), embeddings[1].cpu())
        similarity_scores.append(similarity_score)

    Result = Rouge.compute(predictions=Decoded_Preds, references=Decoded_Labels, use_stemmer=True)

    Prediction_Lens = [np.count_nonzero(Pred != Tokenizer.pad_token_id) for Pred in Predictions]
    Result["gen_len"] = np.mean(Prediction_Lens)
    Result["semantic_similarity"] = np.mean(similarity_scores)

    print({k: round(v, 4) for k, v in Result.items()})

In [134]:
def Test_Model(Paragraph):
    Paragraph_text = Prefix + Paragraph
    Paragraph_ids = Tokenizer(Paragraph_text, max_length=1000, truncation=True, padding="longest", return_tensors="pt").to(Model.device)
    Summary_ids = Model.generate(Paragraph_ids['input_ids'],  num_beams=4, decoder_start_token_id=Model.config.pad_token_id)

    Summary = Tokenizer.batch_decode(Summary_ids, skip_special_tokens=True)

    return Summary, Summary_ids

In [135]:
Record_Index = 0
Paragraph = Test_Data_Frame[Test_Dataset_Paragraphs_Column_Name][Record_Index]
Summary, Summary_ids = Test_Model(Paragraph)

c:\Anaconda\envs\Cuda\lib\site-packages\transformers\generation\utils.py:1313: UserWarning: Using `max_length`'s default (200) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [136]:
Labels = Tokenizer(Test_Data_Frame[Test_Dataset_Summaries_Column_Name][Record_Index], max_length=300, truncation=True, padding="longest", return_tensors="pt").to(Model.device)

In [137]:
Calculate_Metrics(Summary_ids.cpu(), Labels['input_ids'].cpu())

{'rouge1': 0.0, 'rouge2': 0.0, 'rougeL': 0.0, 'rougeLsum': 0.0, 'gen_len': 199.0, 'semantic_similarity': 0.9742}


In [141]:
print("Text: ", Paragraph)
print("Actual Summary: ", Test_Data_Frame[Test_Dataset_Summaries_Column_Name][Record_Index])
print("Predicted Summary: ", Summary[0])
print("Actual Summary Length: ", len(Test_Data_Frame[Test_Dataset_Summaries_Column_Name][Record_Index].split()))
print("Predicted Summary Length: ", len(Summary[0].split()))

Text:  داعش قالت إنها عازمة على الزحف نحو العاصمة العراقية بغداد وصرح مصدر أمني وشهود عيان لبي بي سي في مدينة بعقوبة مركز محافظة ديالى شرقي العراق بأن " مجاميع مسلحة تتقدمها القاعدة بسطت سيطرتها دون قتال على مدينة جلولاء مساء الخميس بعد أن فرت منها قوات الجيش والشرطة". وأضاف المصدر أن " قوات البيشمركة الكردية مكثت في مواقعها في المدينة، ولم تشتبك مع المسلحين". ويسكن مدينة جلولاء خليط من العرب والكرد من السنة والشيعة وتعد ضمن المناطق المتنازع عليها بين الحكومة المركزية وحكومة إقليم كردستان. كما أنها ثاني مدن محافظة ديالى التي سيطر عليها المسلحون الخميس. وأعلن مسلحو (داعش) عزمهم التقدم باتجاه العاصمة بغداد، والمناطق ذات الغالبية الشيعية. مواضيع قد تهمك نهاية وذكرت تقارير غير مؤكدة أن القوات الحكومية شنت غارات جوية في الموصل وتكريت استهدفت المتشددين فيهما. وتمكنت القوات الحكومية من تعطيل تقدم المتشددين في مدينة سامراء، الواقعة على بعد 110 كلم شمالي العاصمة بغداد. احتجاز المئات في هذه الأثناء، بث المسلحون المتشددون مقاطع فيديو على شبكة الانترنت لاحتجاز " المئات من الجنود العراقيين في معسكر

In [ ]:
# Model_Save_Path = 'mBart_707.pt'
# torch.save(Model.state_dict(), Model_Save_Path)

In [ ]:
# Model.load_state_dict(torch.load(Model_Save_Path))

<All keys matched successfully>